In [1]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from apex import amp

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

#Text Cleaning
from bs4 import BeautifulSoup
import re 

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_May__3_19:15:13_PDT_2021
Cuda compilation tools, release 11.3, V11.3.109
Build cuda_11.3.r11.3/compiler.29920130_0


In [3]:
torch.cuda.is_available()

True

In [4]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.synchronize()

In [5]:
CONFIG = {"seed": 2021,
          "epochs": 4,
          "model_name": "GroNLP/hateBERT",
          "train_batch_size": 28,
          "valid_batch_size": 56,
          "max_length": 100,
          "learning_rate": 1e-5,
          "epsilon" : 1e-7,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-5,
          "n_fold": 1,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "patience": 2,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [6]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [7]:
df_train = pd.read_csv('train.csv')
df_train.head()

,txt,offensiveness_score
0,> The difference in average earnings between m...,-0.083
1,"The myth is that the ""gap"" is entirely based o...",-0.022
2,The assertion is that women get paid less for ...,-0.146
3,You said in the OP that's not what they're mea...,-0.083
4,>Men and women are not payed less for the same...,-0.042


In [8]:
df_valid = pd.read_csv("input/jigsaw-toxic-severity-rating/validation_data.csv")
df_valid.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [9]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [10]:
df_train.txt=df_train.txt.apply(str)

In [11]:
df_train.txt = df_train.txt.apply(text_cleaning)
df_valid.less_toxic = df_valid.less_toxic.apply(text_cleaning)
df_valid.more_toxic = df_valid.more_toxic.apply(text_cleaning)

In [12]:
'''group_kfold = GroupKFold(n_splits=5)
for fold, (trn_idx, val_idx) in enumerate(group_kfold.split(df, df, df['group'])): 
    df.loc[val_idx , "kfold"] = fold

df["kfold"] = df["kfold"].astype(int)
df.to_csv('train_noleak.csv', index=False)
df.head()'''

'group_kfold = GroupKFold(n_splits=5)\nfor fold, (trn_idx, val_idx) in enumerate(group_kfold.split(df, df, df[\'group\'])): \n    df.loc[val_idx , "kfold"] = fold\n\ndf["kfold"] = df["kfold"].astype(int)\ndf.to_csv(\'train_noleak.csv\', index=False)\ndf.head()'

In [13]:
class JigsawDatasetTrain(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.txt = df['txt'].values
        self.target = df['offensiveness_score'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        txt = self.txt[index]
        inputs = self.tokenizer.encode_plus(
                                txt,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = self.target
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        
        
        return {
            'ids': torch.tensor(ids, dtype=torch.int),
            'mask': torch.tensor(mask, dtype=torch.int),
            'target': torch.tensor(target, dtype=torch.float)
        }

In [14]:
class JigsawDatasetValid(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.int),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.int),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.int),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.int),
            'target': torch.tensor(target, dtype=torch.int)
        }

In [15]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):
        out = self.model(input_ids=ids,attention_mask=mask,
                             output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        
        return outputs

In [16]:
def criterion_validation(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)

In [17]:
def criterion_train(outputs, targets):
    return nn.MSELoss()(outputs, targets)

In [18]:
def accuracy(y_more_toxic, y_less_toxic): 
    correct = torch.sum(torch.where(y_more_toxic>y_less_toxic, 1, 0))
    wrong = torch.sum(torch.where(y_more_toxic<=y_less_toxic, 1, 0))
    acc = correct / (correct + wrong)
    return acc.view(-1).cpu().detach().numpy()

In [19]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    #Automatic Mixed Precision
    scaler = torch.cuda.amp.GradScaler()
    #------------------
    
    score_batch = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
            
        
        ids = data['ids'].to(device, dtype = torch.int)
        mask = data['mask'].to(device, dtype = torch.int)
        targets = data['target'].to(device, dtype=torch.float)

        batch_size = ids.size(0)

        #Automatic Mixed Precision
        with torch.cuda.amp.autocast():
        #------------------
            outputs = model(ids, mask)
            loss = criterion_train(outputs, targets)
            loss = loss / CONFIG['n_accumulate']
        
        #Automatic Mixed Precision
        scaler.scale(loss).backward()
        #------------------

        if (step + 1) % CONFIG['n_accumulate'] == 0:
                
            #optimizer.step()
            
            #Automatic Mixed Precision
            scaler.step(optimizer)
            scaler.update()
            #------------------
            
            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

 
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    return epoch_loss

In [20]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    score_batch = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.int)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.int)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.int)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.int)
        targets = data['target'].to(device, dtype=torch.int)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion_validation(more_toxic_outputs, less_toxic_outputs, targets)
        score_batch.append(accuracy(more_toxic_outputs, less_toxic_outputs))
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        epoch_score = np.mean(score_batch)
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    return epoch_loss, epoch_score

In [21]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    epochs_no_improve = 0
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Score'].append(val_epoch_score)
        
        print(f"{b_}Valid score {val_epoch_score}")
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            history["Best Loss"].append(best_epoch_loss)
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= CONFIG["patience"]:
                print('Early stopping!' )
                print()
                break
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [22]:
def prepare_loaders(fold):
    #df_train = df[df.kfold != fold].reset_index(drop=True)
    #df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = JigsawDatasetTrain(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = JigsawDatasetValid(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [23]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = JigsawModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], eps=CONFIG['epsilon'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()
    

====== Fold: 0 ======


Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: NVIDIA GeForce RTX 2080



100%|█| 8187/8187 [25:06<00:00,  5.43it/s, Epoch=1, LR=7.24e-6, Train_Loss=0.052
100%|████| 538/538 [03:00<00:00,  2.98it/s, Epoch=1, LR=7.24e-6, Valid_Loss=0.5]


Valid score 0.5555223822593689
Validation Loss Improved (inf ---> 0.49987811226681345)
Model Saved



 97%|▉| 7938/8187 [23:59<00:44,  5.59it/s, Epoch=2, LR=8.66e-6, Train_Loss=0.051